<a href="https://colab.research.google.com/github/Ter20/my_roby/blob/main/mars5_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade vocos encodec librosa safetensors

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 20.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for encodec: filename=encodec-0.1.1-py3-none-any.whl size=45760 sha256=a1f3ec0ac5d9bee5d1075434b838457d8ab388aafecd964af061160eaf1536f1
  Stored in directory: /root/.cache/pip/wheels/fc/36/cb/81af8b985a5f5e0815312d5e52b41263237af07b977e6bcbf3
Successfully built encodec


In [ ]:
import pprint
import IPython.display as ipd
import torch
import librosa

In [ ]:
# load model
mars5, config_class = torch.hub.load('Camb-ai/mars5-tts', 'mars5_english', trust_repo=True)

Now that the model is loaded, pick a reference audio to clone from. If you want to use deep clone, also specify its transcript.

In [ ]:
# download example ref audio
!wget -O example.wav https://github.com/Camb-ai/mars5-tts/raw/master/docs/assets/example_ref.wav

In [ ]:
wav, sr = librosa.load('./example.wav',
                       sr=mars5.sr, mono=True)
wav = torch.from_numpy(wav)
ref_transcript = "We actually haven't managed to meet demand."
print("Reference audio:")
ipd.display(ipd.Audio(wav.numpy(), rate=mars5.sr))
print(f"Reference transcript: {ref_transcript}")

In [ ]:
deep_clone = True # set to False if you don't know prompt transcript or want fast inference.
# Below you can tune other inference settings, like top_k, temperature, top_p, etc...
cfg = config_class(deep_clone=deep_clone, rep_penalty_window=100,
                      top_k=100, temperature=0.7, freq_penalty=3)

ar_codes, wav_out = mars5.tts("The quick brown rat.", wav,
          ref_transcript,
          cfg=cfg)

print('Synthesized output audio:')
ipd.Audio(wav_out.numpy(), rate=mars5.sr)

You can see all the inference settings available to tune in the inference config here:

In [ ]:
pprint.pprint(config_class())

You can also listen to the vocoded raw coarse codes, for debugging purposes:

In [ ]:
ar_wav = mars5.vocode(ar_codes.cpu()[:, None])
ipd.Audio(ar_wav.numpy(), rate=mars5.sr)